In [ ]:
# Scripting Language : Python

# Libraries : psycopg2 for PostgreSQL database interaction
#             SQLAlchemy for creating a database engine.
#             Pandas for data manipulation

# Approach 
# Step1 : Create a PostgreSQL database
# Step2 : Connect Python NoteBook with the Database
# Step3 : Create Required Tables and Import the data from CSV file
# Step4 : Perform the required analysis


In [ ]:
# Importing Libraries
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from psycopg2.extras import execute_batch

In [ ]:
# Reading data from ratings.csv
csv_file_path = 'ratings.csv'
df = pd.read_csv(csv_file_path)

In [ ]:
# Defining Connection Parameters
db_connection = {
    'host': 'localhost',
    'port': '5432',
    'database': 'postgres',
    'user': 'postgres',
    'password': '1234'
}

In [ ]:
df['time'] = pd.to_datetime(df['time'], unit='s', origin='unix', utc=True)
table_name = 'ratings'
engine = create_engine(f'postgresql+psycopg2://{db_connection["user"]}:{db_connection["password"]}@{db_connection["host"]}:{db_connection["port"]}/{db_connection["database"]}')

with engine.connect() as con:
    con.execute("SET datestyle = 'ISO';")

df.to_sql(table_name, con=engine, if_exists='replace', index=False)

engine.dispose()

In [ ]:
#Reading data from movies.csv
csv_file_path = 'movies.csv'
df = pd.read_csv(csv_file_path)


# Since 'country' and 'genre' have comma seperated values in them 
# I converted CSVs in the 'country' and 'genre' columns to arrays
df['country'] = df['country'].apply(lambda x: x.split(',') if isinstance(x, str) else [])
df['genre'] = df['genre'].apply(lambda x: x.split(',') if isinstance(x, str) else [])

# Establish a connection to PostgreSQL
conn = psycopg2.connect(**db_connection)

# Open a cursor to perform database operations
cursor = conn.cursor()


# Creating the movies table 
create_table_query = '''
CREATE TABLE IF NOT EXISTS movies (
    id INTEGER PRIMARY KEY,
    title VARCHAR(500),  
    year INTEGER,
    country VARCHAR(255)[] NOT NULL,
    genre VARCHAR(255)[] NOT NULL,
    director VARCHAR(500),  
    minutes INTEGER,
    poster VARCHAR(1000)  
);
'''
cursor.execute(create_table_query)


# Inserting data into the movies table using execute_batch
insert_query = '''
INSERT INTO movies (id, title, year, country, genre, director, minutes, poster)
VALUES (%(id)s, %(title)s, %(year)s, %(country)s, %(genre)s, %(director)s, %(minutes)s, %(poster)s);
'''

# Converting DataFrame to a list of dictionaries
data = df.to_dict(orient='records')

# Execute the query in batches
execute_batch(cursor, insert_query, data, page_size=1000)  

conn.commit()


In [27]:
cursor.execute('''
    SELECT title, minutes
    FROM movies
    ORDER BY minutes DESC
    LIMIT 5;
''')
result_duration = cursor.fetchall()
print("Top 5 Movie Titles by Duration:")
print(pd.DataFrame(result_duration, columns=['Title', 'Duration']))
print()

Top 5 Movie Titles by Duration:
                         Title  Duration
0           Gangs of Wasseypur       320
1           Gone with the Wind       238
2  Once Upon a Time in America       229
3           Lawrence of Arabia       216
4                 Jodhaa Akbar       213



In [28]:
# Top 5 Movie Titles by Year of Release
cursor.execute('''
    SELECT title, year
    FROM movies
    ORDER BY year DESC
    LIMIT 5;
''')
result_year = cursor.fetchall()
print("Top 5 Movie Titles by Year of Release:")
print(pd.DataFrame(result_year, columns=['Title', 'Year']))
print()

Top 5 Movie Titles by Year of Release:
                     Title  Year
0          Bridge of Spies  2015
1           Irrational Man  2015
2                    Ted 2  2015
3           McFarland, USA  2015
4  Cobain: Montage of Heck  2015



In [29]:
# Top 5 Movie Titles by Average Rating (considering movies with minimum 5 ratings)
cursor.execute('''
    SELECT title, AVG(rating) AS average_rating
FROM movies
JOIN ratings ON movies.id = ratings.movie_id
GROUP BY movies.id, title
HAVING COUNT(ratings.rater_id) >= 5
ORDER BY average_rating DESC
LIMIT 5;
''')
result_avg_rating = cursor.fetchall()
print("Top 5 Movie Titles by Average Rating:")
print(pd.DataFrame(result_avg_rating, columns=['Title', 'Average Rating']))
print()

Top 5 Movie Titles by Average Rating:
                                            Title      Average Rating
0                               L.A. Confidential  9.8000000000000000
1                          The Godfather: Part II  9.8000000000000000
2                        The Shawshank Redemption  9.6500000000000000
3   The Lord of the Rings: The Return of the King  9.6000000000000000
4  Star Wars: Episode V - The Empire Strikes Back  9.5714285714285714



In [30]:
# Top 5 Movie Titles by Number of Ratings Given
cursor.execute('''
    SELECT title, COUNT(ratings.rater_id) AS rating_count
    FROM movies
    LEFT JOIN ratings ON movies.id = ratings.movie_id
    GROUP BY movies.id, title
    ORDER BY rating_count DESC
    LIMIT 5;
''')
result_rating_count = cursor.fetchall()
print("Top 5 Movie Titles by Number of Ratings Given:")
print(pd.DataFrame(result_rating_count, columns=['Title', 'Rating Count']))
print()

Top 5 Movie Titles by Number of Ratings Given:
                     Title  Rating Count
0  The Wolf of Wall Street            69
1                  Gravity            64
2             Man of Steel            60
3           Now You See Me            59
4             Interstellar            58



In [31]:
cursor.execute('''
    SELECT COUNT(DISTINCT rater_id) AS unique_rater_count
    FROM ratings;
''')
result_unique_raters = cursor.fetchone()
print("Number of Unique Raters:", result_unique_raters[0])

Number of Unique Raters: 1048


In [33]:
# Top 5 Rater IDs by Most Movies Rated
cursor.execute('''
    SELECT rater_id, COUNT(*) AS movies_rated_count
    FROM ratings
    GROUP BY rater_id
    ORDER BY movies_rated_count DESC
    LIMIT 5;
''')
result_most_movies_rated = cursor.fetchall()
print("Top 5 Rater IDs by Most Movies Rated:")
print(pd.DataFrame(result_most_movies_rated, columns=['Rater ID', 'Movies Rated Count']))
print()

# Top 5 Rater IDs by Highest Average Rating Given (consider raters with a minimum of 5 ratings)
cursor.execute('''
    SELECT rater_id, AVG(rating) AS average_rating
    FROM ratings
    GROUP BY rater_id
    HAVING COUNT(*) >= 5
    ORDER BY average_rating DESC
    LIMIT 5;
''')
result_highest_avg_rating = cursor.fetchall()
print("Top 5 Rater IDs by Highest Average Rating:")
print(pd.DataFrame(result_highest_avg_rating, columns=['Rater ID', 'Average Rating']))
print()
print()

Top 5 Rater IDs by Most Movies Rated:
   Rater ID  Movies Rated Count
0       735                 314
1       769                 209
2       512                 188
3       800                 186
4        82                 159

Top 5 Rater IDs by Highest Average Rating:
   Rater ID       Average Rating
0       206  10.0000000000000000
1       256  10.0000000000000000
2       231  10.0000000000000000
3       662  10.0000000000000000
4       146  10.0000000000000000




In [37]:
# Top Rated Movie by Director 'Michael Bay'
cursor.execute('''
    SELECT movies.title, AVG(ratings.rating) AS average_rating
    FROM movies
    JOIN ratings ON movies.id = ratings.movie_id
    WHERE movies.director = 'Michael Bay'
    GROUP BY movies.id, movies.title
    HAVING COUNT(ratings.rater_id) >= 5
    ORDER BY average_rating DESC
    LIMIT 1;
''')
result_director = cursor.fetchall()
print("Top Rated Movie by Director 'Michael Bay':")
print(pd.DataFrame(result_director, columns=['Title', 'Average Rating']))
print()

Top Rated Movie by Director 'Michael Bay':
          Title      Average Rating
0  Transformers  8.6000000000000000



In [38]:
# Top Rated Comedy Movie
cursor.execute('''
    SELECT movies.title, AVG(ratings.rating) AS average_rating
    FROM movies
    JOIN ratings ON movies.id = ratings.movie_id
    WHERE 'Comedy' = ANY(movies.genre)
    GROUP BY movies.id, movies.title
    HAVING COUNT(ratings.rater_id) >= 5
    ORDER BY average_rating DESC
    LIMIT 1;
''')
result_comedy = cursor.fetchall()
print("Top Rated Comedy Movie:")
print(pd.DataFrame(result_comedy, columns=['Title', 'Average Rating']))
print()

Top Rated Comedy Movie:
  Title      Average Rating
0    PK  9.3000000000000000



In [39]:
# Top Rated Movie in the Year 2013
cursor.execute('''
    SELECT movies.title, AVG(ratings.rating) AS average_rating
    FROM movies
    JOIN ratings ON movies.id = ratings.movie_id
    WHERE movies.year = 2013
    GROUP BY movies.id, movies.title
    HAVING COUNT(ratings.rater_id) >= 5
    ORDER BY average_rating DESC
    LIMIT 1;
''')
result_year = cursor.fetchall()
print("Top Rated Movie in the Year 2013:")
print(pd.DataFrame(result_year, columns=['Title', 'Average Rating']))
print()

Top Rated Movie in the Year 2013:
           Title      Average Rating
0  Short Term 12  8.7777777777777778



In [40]:
# Top Rated Movie in India
cursor.execute('''
    SELECT movies.title, AVG(ratings.rating) AS average_rating
    FROM movies
    JOIN ratings ON movies.id = ratings.movie_id
    WHERE 'India' = ANY(movies.country)
    GROUP BY movies.id, movies.title
    HAVING COUNT(ratings.rater_id) >= 5
    ORDER BY average_rating DESC
    LIMIT 1;
''')
result_country = cursor.fetchall()
print("Top Rated Movie in India:")
print(pd.DataFrame(result_country, columns=['Title', 'Average Rating']))
print()

Top Rated Movie in India:
  Title      Average Rating
0    PK  9.3000000000000000



In [41]:
# Favorite Movie Genre of Rater ID 1040
cursor.execute('''
    SELECT genre
    FROM (
        SELECT genre, COUNT(*) AS genre_count
        FROM movies
        JOIN ratings ON movies.id = ratings.movie_id
        WHERE ratings.rater_id = 1040
        GROUP BY genre
        ORDER BY genre_count DESC
        LIMIT 1
    ) AS favorite_genre;
''')
result_favorite_genre = cursor.fetchone()

if result_favorite_genre:
    print(f"Favorite Movie Genre of Rater ID 1040: {result_favorite_genre[0]}")
else:
    print("Rater ID 1040 has not rated any movies.")

Favorite Movie Genre of Rater ID 1040: ['Action', ' Adventure', ' Sci-Fi']


In [48]:
# Highest Average Rating for a Movie Genre by Rater ID 1040
cursor.execute('''
    SELECT genre, MAX(average_rating) AS highest_average_rating
    FROM (
        SELECT genre, AVG(rating) AS average_rating
        FROM movies
        JOIN ratings ON movies.id = ratings.movie_id
        WHERE ratings.rater_id = 1040
        GROUP BY genre
        HAVING COUNT(ratings.rater_id) >= 5
    ) AS genre_ratings
    GROUP BY genre;
''')
result_highest_avg_rating = cursor.fetchone()

if result_highest_avg_rating:
    print(f"Highest Average Rating for a Movie Genre by Rater ID 1040:")
    print(f"Genre: {result_highest_avg_rating[0]}, Highest Average Rating: {result_highest_avg_rating[1]}")
else:
    print("Rater ID 1040 has not rated any movies in genres with a minimum of 5 ratings.")


Highest Average Rating for a Movie Genre by Rater ID 1040:
Genre: ['Action', ' Adventure', ' Sci-Fi'], Highest Average Rating: 8.4545454545454545


In [51]:
# Year with Second-Highest Number of Action Movies
cursor.execute('''
    SELECT year, COUNT(*) AS action_movie_count
FROM movies
JOIN ratings ON movies.id = ratings.movie_id
WHERE 'Action' = ANY(movies.genre)
  AND 'USA' = ANY(movies.country)
GROUP BY year
HAVING AVG(ratings.rating) >= 6.5
  AND MAX(movies.minutes) < 120
ORDER BY action_movie_count DESC
OFFSET 1
LIMIT 1;
''')
result_second_highest_action_count = cursor.fetchone()

if result_second_highest_action_count:
    print(f"Year with Second-Highest Number of Action Movies:")
    print(f"Year: {result_second_highest_action_count[0]}, Action Movie Count: {result_second_highest_action_count[1]}")
else:
    print("No qualifying movies found.")

Year with Second-Highest Number of Action Movies:
Year: 1984, Action Movie Count: 4


In [54]:
# Count of Movies with High Ratings
cursor.execute('''
    SELECT COUNT(DISTINCT movies.id) AS high_rated_movie_count
    FROM movies
    JOIN ratings ON movies.id = ratings.movie_id
    WHERE ratings.rating >= 7
    GROUP BY movies.id
    HAVING COUNT(ratings.rater_id) >= 5;
''')
result_high_rated_movie_count = cursor.fetchone()

if result_high_rated_movie_count:
    print(f"Number of Movies with High Ratings (at least five reviews with a rating of 7 or higher):")
    print(f"Count: {result_high_rated_movie_count[0]}")
else:
    print("No movies meet the specified criteria.")

Number of Movies with High Ratings (at least five reviews with a rating of 7 or higher):
Count: 1


In [56]:
cursor.close()
conn.close()